In [1]:
from keras.applications import VGG16

# VGG16 합성곱 기반 층 만들기
conv_base = VGG16(weights = "imagenet", include_top = False, input_shape = (150, 150, 3))
# weights : 모델을 초기화 할 가중치 체크포인트를 지정한다.
# include_top : 네트워크의 최상위 완전 연결 분류기를 포함할지 안 할지 지정한다.
# input_shaep : 네트워크에 주입할 이미지 텐서의 크기
conv_base.summary()

Using TensorFlow backend.


Instructions for updating:
Colocations handled automatically by placer.
  720896/58889256 [..............................] - ETA: 29:41:42

KeyboardInterrupt: 

In [2]:
# 데이터 증식을 사용하지 않는 빠른 특성 추출
# 사전 훈련된 합성곱 기반 층을 사용한 특성 추출하기
import os
import numpy as np
from keras.preprocessing.image import ImageDataGenerator

base_dir = "cats_and_dogs"
train_dir = os.path.join(base_dir, "train")
validation_dir = os.path.join(base_dir, "validation")
test_dir = os.path.join("test")

datagen = ImageDataGenerator(rescale = 1. / 255)
batch_size = 20

def extract_features(directory, sample_count):
    features = np.zeros(shape = (sample_count, 4, 4, 512))
    labels = np.zeros(shape = (sample_count))
    generator = datagen.flow_from_directory(directory, target_size = (150, 150), batch_size = batch_size, class_mode = "binary")
    
    i = 0
    
    for inputs_batch, labels_batch in generator:
        features_batch = conv_base.predict(inputs_batch)
        features[i * batch_size : (i + 1) * batch_size] = features_batch
        labels[i * batch_size : (i + 1) * batch_size] = labels_batch
        i += 1
        if i * batch_size >= sample_count:
            break
            
    return features, labels

train_features, train_labels = extract_features(train_dir, 2000)
validation_features, validation_labels = extract_features(validation_dir, 1000)
test_features, test_labels = extract_features(test_dir, 1000)

# 추출된 특성의 크기는 (samples, 4, 4, 512)이며, 완전 연결 분류기에 주입하기 위해 (samples, 8192) 크기로 펼친다.
train_features = np.reshape(train_features, (2000, 4 * 4 * 512))
validation_features = np.reshape(validation_features, (1000, 4 * 4 * 512))
test_features = np.reshape(test_features, (1000, 4 * 4 * 512))

Found 2000 images belonging to 2 classes.


NameError: name 'conv_base' is not defined

In [ ]:
# 완전 연결 분류기를 정의하고 훈련하기
from keras import models
from keras import layers
from keras import optimizers

model = models.Sequential()
model.add(layers.Dense(256, activation = "relu", input_dim = 4 * 4 * 512))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(1, activation = "sigmoid"))

model.compile(optimizer = optimizers.RMSprop(lr = 2e-5), loss = "binary_crossentropy", metrics = ["acc"])

history = model.fit(train_features, train_labels, epochs = 5, batch_size = 20, 
                    validation_data = (validation_features, validation_labels))

In [ ]:
# 결과 그래프 그리기
import matplotlib.pyplot as plt

acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(acc) + 1)

plt.plot(epochs, acc, "bo", label = "Training acc")
plt.plot(epochs, val_acc, "b", label = "Validation acc")
plt.title("Training and Validation accuracy")
plt.legend()

plt.figure()

plt.plot(epochs, loss, "bo", label = "Training loss")
plt.plot(epochs, val_loss, "b", label = "Validation loss")
plt.title("Training and Validation loss")
plt.legend()

plt.show()